In [13]:
import pandas as pd
import numpy as np
import cobra
import re
from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.fastcore import FASTcore, FastcoreProperties
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.gapfill.fastcc import FastCC, FastCCProperties
from cobra.io import read_sbml_model
import numpy as np

# parsing rule 
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')

expression_data = pd.read_csv('/home/biodata/aman/data/data_processed/expression_data_hippocampus-development-human-brain-10XV2_annotated_normalized_annotated_gencode.csv', index_col=0)

expression_data_T = expression_data.T # transpose

omics_container = TabularReader(path_or_df=expression_data_T, nomenclature='gene',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]

model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)
from math import log
threshold =  (50 * log(2))

keywords = [
    "oxidative", "phosphorylation",
    "taurine", 
    "glycerophospholipid",
    "glutamate", 
    "glutamine", "aspartate", 
    "malate", 
    "glycine", "serine", "threonine",
    "cholesterol", 
    "sphingolipid"
] # for brain hippocampus dev. ## doi: 10.1021/acschemneuro.5c00006

rid_lists = []

for kw in keywords:
    for r in model.reactions:
        if r.subsystem and kw in r.subsystem.lower():
            rid_lists.append(r.id)

print(rid_lists)

protected_reactions = rid_lists # switch to the real rid. 

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)

print(threshold_scores)
# Get the index of the reaction of the CORE reaction set
ordered_ids = {r:i for i,r in enumerate(model_wrapper.model_reader.r_ids)}
core_idx = [[ordered_ids[k] for k in l] for l in threshold_scores]

# Define the FastCORE properties
properties = FastcoreProperties(core=core_idx, solver='CPLEX')

# instantiate the FastCORE class
fastcore = FASTcore(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

# Run the algorithm
model_fastcore = fastcore.run()

/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000147576 or _ENSG00000172955 or _ENSG00000180011 or _ENSG00000187758 or _ENSG00000196344 or _ENSG00000196616 or _ENSG00000197894 or _ENSG00000198099 or _ENSG00000248144
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000117448
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000131069
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111058 or _ENSG00000154930
  warnings.war

[['MAR03905', 'MAR03907', 'MAR04281', 'MAR04388', 'MAR08357', 'MAR04379', 'MAR04301', 'MAR04355', 'MAR04358', 'MAR04363', 'MAR04365', 'MAR04368', 'MAR04370', 'MAR04371', 'MAR04372', 'MAR04373', 'MAR04375', 'MAR04381', 'MAR04391', 'MAR04394', 'MAR08360', 'MAR08585', 'MAR03944', 'MAR04130', 'MAR04414', 'MAR04416', 'MAR08762', 'MAR08766', 'MAR04316', 'MAR04317', 'MAR04318', 'MAR04319', 'MAR04320', 'MAR04356', 'MAR04385', 'MAR04399', 'MAR04490', 'MAR04706', 'MAR04590', 'MAR04592', 'MAR08341', 'MAR08344', 'MAR08727', 'MAR06537', 'MAR03853', 'MAR03854', 'MAR03855', 'MAR03857', 'MAR04087', 'MAR04091', 'MAR04193', 'MAR08497', 'MAR08500', 'MAR08501', 'MAR08502', 'MAR08503', 'MAR08504', 'MAR08506', 'MAR08507', 'MAR08508', 'MAR08509', 'MAR08511', 'MAR04280', 'MAR03212', 'MAR04741', 'MAR09804', 'MAR00719', 'MAR01434', 'MAR03163', 'MAR04604', 'MAR05351', 'MAR04404', 'MAR04473', 'MAR04474', 'MAR04501', 'MAR04565', 'MAR04567', 'MAR04625', 'MAR08653', 'MAR03998', 'MAR04000', 'MAR04002', 'MAR04004', 'M

Exception: Inconsistent irreversible core reactions 
	Impossible to build model